In [27]:
import numpy as np
from time import time
from Crypto.Util.number import getPrime

질문!! e는 계속 바뀌어야 안전한가요??

## 0. 최종 결과 정리

### 1) encoding 방법

In [ ]:
# 우선 p,q의 크기는 1024비트 정수로 하였으며, 메세지의 인코딩 시에는 e*2^25의 에러 텀을
# 삽입하였으며, e는 64비트 소수로 설정하였습니다.

### 2) 구현 test vector

In [ ]:
#x,y 두개의 벡터를 구현하였으며, 각각의 길이는 문제에서 요구된 대로 256 입니다.
# 각 벡터에는 평균이 50이고 std=20인 정규 분포에 따르는 랜덤값중 0~100 범위에 있는
# 값들을 추출하였습니다.

### 3) 계산 속도 및 컴퓨터 사양

In [ ]:
# 프로세서는 Intel(R) Core(TM) i7-7700HQ CPU 2.8GHz 입니다.
# 암호화에 소요되는 시간은 0.0014801025390625 이며, 
# 덧셈의 암호화된 연산, 복호화 과정을 합쳐서  0.0004930496215820312 가 소요되었고, 
# 분산의 암호화된 연산, 복호화 과정을 합쳐서 0.0015254020690917969 가 소요되었습니다
# 전체적으로 거의 시간이 들지 않았다고 할 수 있습니다.

### 4) 응용점 제안

## 1. 암호화, 복호화 함수 구현

In [125]:
### extended euclidean algorithm
# 암호화 과정에서 필수적인 extended euclidean algorithm을 먼저 구현
def EEA(a,b):
    
    r_i, r_i_new = (a, b) 

    s_i, s_i_new = (1,0)
    t_i, t_i_new = (0,1)
    
    while r_i_new !=0:
        q = r_i // r_i_new
        s_i, s_i_new = (s_i_new, s_i - q*s_i_new)
        t_i, t_i_new = (t_i_new, t_i - q*t_i_new)
        r_i, r_i_new = (r_i_new, r_i - q*r_i_new)
        
    return r_i, s_i, t_i

### ENCRYPTION function
def HE_encryption(m1, m2, p, q):
    
    # known plaintext attack에서 암호체계를 보호하기 위해 noise term  e를 메세지에 삽입
    # 이 결과 이진법 단위에서 25bit의 noise term + 3bit 여유 + message의 22bit가 됨
    # 22bit인 이유는, 분산 계산시  256(=2^8)개의 2^7bit의  개체의 제곱합이 계산되므로 
    # 최대 7X2 +8 = 22 bit의 공간을 차지 
    x_i = (2**25)*e + m1
    y_i = (2**25)*e + m2
    
    # extended euclidean algorithm을 통해 p*p_ + q*q_ = 1 인 p_. q_ 를 찾습니다.
    # r=1 이여야 하므로 만에 하나 r!=1 일때는 에러를 반환하게 하였습니다.
    r, p_, q_ = EEA(p,q)
    
    if r !=1:
        raise ValueError("GCD of P,Q is not 1 try another p and q")
    
    #본격적으로 chinese remainder theorem을 사용해 값을 암호화 합니다.
    CRT = (x_i*q_*q + y_i*p_*p) % (p*q)
    
    return CRT


### DECRYPTION function
def HE_decryption(crt, p, q):
    
    # chinese remainder theorem을 이용한 복호화 과정
    x_i = crt  % p
    y_i = crt  % q
    
    # noise term e*2^25 를 삭제해주어 원문 메세지를 복원하는 과정
    m1 = x_i  % (2**25)
    m2 = y_i % (2**25)
    
    
    return m1, m2

## 2. sum, variance 함수 구현

In [145]:
# 벡터 값들의 합과 분산을 계산하는 함수를 정의해 줍니다
def sum_(score_list):
    sum_of_score= 0
    for i in score_list:
        sum_of_score += i
    
    return sum_of_score
    
def sum_of_square_minus_mean(score_list):
    sum_of_squares = 0
    for i in score_list:
        sum_of_squares += i**2
    
    final_value = sum_of_squares  
    
    return final_value

## 3. 비밀키 및 실험용 데이터셋 생성 

In [146]:
### 평균과 분산을 계산할 두개의 벡터를 생성 
#normal분포에, 평균 50이고 std는 20인 random값들 중 0~100 사이의 값들을 256개 뽑습니다.  
score_vector = []

def score_vector_generator():
    score_vector = []
    for i in range(2**9):
        score = round(np.random.normal(loc = 50, scale = 20))
        if score<=100 and score>=0:
            score_vector.append(score)
        if len(score_vector) == 2**8:
            break
    return np.array(score_vector)

vector_m1 = score_vector_generator()
vector_m2 = score_vector_generator()

In [147]:
### 비밀키 p,q ,e 생성.
#이때 p,q는 message 자리수 + noise term 자리수를 가지고 연산한 값의 최대 자리수 보다 
# 커야 하는데, 넉넉하게 2^1024 정도로 잡았습니다.  
#동형암호를 이용한 과정에서는 암호화와 복호화 모두 한쪽에서 하기에
# 굳이 공개키 n= p*q를 변수로 만들어 저장할 필요는 없어 보입니다.
p= getPrime(1024)
q = getPrime(1024)

# 암호 체계를 known text attack으로 부터 보호하기 위한,
# 적절한 크기의 noise term을 설정해 줍니다. 
e = getPrime(64)
e

11858151691045153759

## 4. 암호화된 sum, variance 연산 실제 구동

In [171]:

# 메세지 암호화
encrypted_message = HE_encryption(vector_m1, vector_m2, p, q)


### 4-1) encrypted sum function 

In [164]:

#암호화된 상태로 덧셈 수행
encrypted_sum = sum_(encrypted_message)


In [167]:

#복호화 과정 수행
original_sum = HE_decryption(encrypted_sum, p,q)


In [151]:
# 덧셈 연산 보존을 확인
original_sum == (np.sum(vector_m1), np.sum(vector_m2))

True

In [152]:
# 덧셈한 값 확인
print(original_sum)

(12664, 12857)


### 4-2) encrypted variance function

In [153]:
# sum of square에 n*평균의 계산을 보존한 것을 확인
encrypted_SSMM = sum_of_square_minus_mean(encrypted_message)

In [154]:
original_SSMM = HE_decryption(encrypted_SSMM,p, q)

In [155]:
# variance를 직접적으로 계산하는 부분
original_variance = np.array(original_SSMM)/n - (np.array(original_sum)**2)/n**2 

In [159]:
# 분산 연산 보존을 확인
original_variance == (np.var(vector_m1) , np.var(vector_m2))

array([ True,  True])

In [160]:
# 분산 값 확인
print(original_variance)

[319.26464844 327.21214294]
